## Step 1
Load model response

In [1]:
import json

# Path to your JSONL file
file_path = "data4verification/iGSM_verify_50_cot.jsonl"
response_list = []
gt_response_list = []

with open(file_path, 'r') as file:
    for line in file:
        obj = json.loads(line)  # Parse each line as a JSON object
        response_list.append(obj)
        # print(obj.keys())  # Process or print each object
        # print(obj['idx'])
        
gt_cot_filepath = "data4verification/iGSM_verify_50.jsonl"
with open(gt_cot_filepath, 'r') as file:
    for line in file:
        obj = json.loads(line)  # Parse each line as a JSON object
        gt_response_list.append(obj)
        # print(obj.keys())  # Process or print each object
        # print(obj['idx'])



## Step 2
Calculate PRM based on responses and DAGs.

In [2]:
import random
import numpy as np
from igsm_gym.generation import ProblemGenerator
from igsm_gym.utils.misc import seed_all
from igsm_gym.env import RewardModel
import json
import copy

config = {
    "english_path": "igsm_gym/generation/english/categorization.json",
    "max_operations": 21,
    "force": False,
}

seed = 10
seed_all(seed)
print(seed)

test_num = 50
data = []
Gd_list = []

pg = ProblemGenerator(config=config,debug=True)

for i in range(test_num):
    while not pg.draw_question():
        continue
    print("index:", i)
    
    question = pg.generate_question()
    answer = pg.generate_answer()
    gt_cot = answer
    gt_answer = pg.Gd.topo[-1].value
    
    response = response_list[i]['code'][0]
    
    reward_model = RewardModel(Gd=pg.Gd)
    PRM_reward = reward_model.PRM(
        llm_output=response
    )
    
    
    # if i == 6:

        
    print(question)
    print("@"*10)
    print("gt cot:")
    print(gt_cot)
    print("gt answer:")
    print(gt_answer)
    print("@"*10)
    print("respones:")
    print(response)
    print("@"*10)
    print("PRM:")
    print(PRM_reward)
    print("="*30)

    
    
# Save the data to a JSONL file
# output_file = "iGSM_verify_{}.jsonl".format(test_num)
# with open(output_file, "w") as f:
#     for entry in data:
#         f.write(json.dumps(entry) + "\n")  # Convert each entry to a JSON string and write it as a new line

10
index: 0
The number of each Z1's Y8 equals the sum of each Y2's X8 and each Y0's X6. The number of each Y0's X9 equals the sum of each Y2's X8 and each Y8's X3. The number of each Z7's Y0 equals each Z7's Y8. The number of each Y8's X3 equals 19 more than each Z0's Y7. The number of each Z5's Y7 equals each Z1's YYY. The number of each Z0's Y2 equals 21 more than each Z5's YYY. The number of each Y2's X8 equals 11. The number of each Z7's Y8 equals each Y0's X9. The number of each Z0's Y7 equals 3 more than each Y2's X8. The number of each Y0's X6 equals the difference of each Y8's X3 and each Z7's Y0. The number of each Z1's Y2 equals each Y0's X9. How many Y2 does each Z0 have?
@@@@@@@@@@
gt cot:
Define each Y2's X8 as l. So l = 11.
Define each Z0's Y7 as d. So d = 3 + l = 3 + 11 = 14.
Define each Y8's X3 as G. So G = 19 + d = 19 + 14 = 10.
Define each Y0's X9 as D. So D = l + G = 11 + 10 = 21.
Define each Z7's Y8 as c. So c = D = 21.
Define each Z7's Y0 as M. So M = c = 21.
Defin

In [3]:
from igsm_gym.generation import ProblemGenerator
from igsm_gym.env import RewardModel

verified_idx = 1
ver_Gd = Gd_list[verified_idx]
ver_response = response_list[verified_idx]['code'][0]
gt_cot = gt_response_list[verified_idx]['gt_cot']
# print(ver_response)

reward_model = RewardModel(Gd=ver_Gd)
PRM_reward = reward_model.PRM(
    llm_output=ver_response
)

print("gt cot is: \n{}".format(gt_cot))
print("="*20)
print("llm output is: \n{}".format(ver_response))
print("="*20)
print("The gt PRM is: \n{}".format(PRM_reward))



GT_final step: 0
gt cot is: 
Define each Y2's X5 as q. So q = 12.
Define each Y2's XXX as S. So S = q = 12.
Define each X5's W9 as k. So k = 22 * S = 22 * 12 = 11.
Define each X5's WWW as C. So C = k = 11.
Define each Y2's WWW as H. So H = q * C = 12 * 11 = 17.
Define each Y4's X5 as F. So F = q + H = 12 + 17 = 6.
Thus, the answer is 6.
llm output is: 
Y2's X5 as u. So u = 12.
Define each Y2's XXX as S. So S = u = 12.
Define each X5's W9 as A. So A = 22 * S = 22 * 12 = 11.
Define each X5's WWW as i. So i = A = 11.
Define each Y2's WWW as Y. So Y = u * i = 12 * 11 = 17.
Define each Y4's X5 as F. So F = u + Y = 12 + 17 = 6.
Thus, the answer is 6.
The gt PRM is: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
